In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from ipywidgets import widgets
import seaborn as sns
from scipy.signal import find_peaks
from time import time
import feather
import xlsxwriter 

# Data Cleansing

In [11]:
# data available in local repository only for privacy issues
def dtfr():
    df = pd.read_excel("Excels/Consumo horario Zonas de carga de la GCROC 2015-2019.xlsx", sheet_name="datos")
    return df

In [12]:
# dtfr().head()

##### Multiple level header on file: remove first header and change space separated names on second header

In [13]:
def lim_header():
    dfi = dtfr()
    dfi.columns = dfi.iloc[0]
    dfi = dfi.drop(index=0)
    dfi.rename(columns = {"día de la semana" : "día_semana"}, inplace=True )
    dfi.rename(columns = {pd.NaT : "fechas"}, inplace = True)
    return dfi

In [14]:
# lim_header().columns

##### Fill on empty day of week and date values

In [15]:
def lim_relleno():
    df = lim_header()
    dia_sem = df["día_semana"].tolist()
    fech = df["fecha"].tolist()
    hor = df["hora"].tolist()
    indx = 0
    for h in hor:
        if h == 1:
            fecha = fech[indx]
            diaa = dia_sem[indx]
            indx += 1
        elif h != 1:
            fech[indx] = fecha
            dia_sem[indx] = diaa
            indx += 1
    df["fecha"] = fech
    df["día_semana"] = dia_sem
    return df

#### Replacement of non-numeric values

In [16]:
def lim_Nnum():
    df = lim_relleno()
    df['year'] = pd.DatetimeIndex(df["fechas"]).year
    df['month'] = pd.DatetimeIndex(df["fechas"]).month
    df['day'] = pd.DatetimeIndex(df["fechas"]).day

    df = df.replace(["Lunes", "Martes", "Miercoles", "Jueves", "Viernes", "Sabado", "Domingo"], [0, 1, 2, 3, 4, 5, 6])
    df = df.replace(["Miércoles", "Sábado"],[2,5])

    df = df.drop(["fecha","fechas"], axis=1)
    df = df.dropna()
    
    return df

#### Change columns names for readability

In [17]:
def lim_nomzon():
    df = lim_Nnum()
    zonas = df.columns
    elim = ["día_semana", "hora", "day", "year", "month"]
    zonas = zonas.drop(elim)
    son = ["zona_{}".format(i+1) for i in range(len(zonas))]
    listDic={}
    for zona, so in zip(zonas, son):
        listDic[zona]=so
    df.rename(columns = listDic, inplace=True )
    return df

###### Save data in a pickle file

In [18]:
lim_nomzon().to_pickle("dflimpios.pkl")